In [ ]:
# === Monta Google Drive ===
from google.colab import drive
drive.mount('/content/drive')

# === Importa funzione get_all_tickers da my_tickers.py ===
import sys
sys.path.append('/content/drive/MyDrive/ColabNotebooks1')
from my_tickers import get_all_tickers

import pandas as pd
import yfinance as yf
import time
import warnings
from datetime import datetime
import os

warnings.filterwarnings("ignore")

# 1. Ottieni lista tickers
tickers = get_all_tickers()  # --> lista ['AAPL', 'MSFT', ...]

# 2. Scarico dati fondamentali con yfinance
data = {}
meta = {}
for t in tickers:
    try:
        ticker = yf.Ticker(t)
        info = ticker.info
        ev = info.get('enterpriseValue', None)
        sales = info.get('totalRevenue', None)   # ultimo anno contabile
        industry = info.get('industry', None)    # settore
        if ev and sales and sales > 0 and industry:
            data[t] = ev / sales
            meta[t] = industry
        time.sleep(0.1)  # evita troppe richieste insieme
    except Exception:
        continue

# 3. Creo DataFrame EV/Sales + Industry
df = pd.DataFrame.from_dict(data, orient="index", columns=["EV/Sales"])
df["Industry"] = pd.Series(meta)

# 4. Calcolo media EV/Sales e numero aziende per industry
result = (
    df.groupby("Industry")
      .agg(
          mean_EV_Sales=("EV/Sales", "mean"),
          num_companies=("EV/Sales", "count")
      )
      .sort_values("mean_EV_Sales", ascending=False)
)

# 5. Mostro risultati
pd.set_option("display.max_rows", None)
print(result)

# 6. Salvataggio file Excel
last_year = datetime.now().year - 1  # ultimo anno chiuso
output_dir = "/content/drive/MyDrive/EV-SALES"
os.makedirs(output_dir, exist_ok=True)

file_path = os.path.join(output_dir, f"EV_Sales_{last_year}.xlsx")

# salvo solo il risultato aggregato
result.to_excel(file_path, engine="openpyxl")

print(f"\n✅ File salvato in: {file_path}")